In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import os
import joblib

# Load the RandomForestClassifier model
model_filename = "hand_gesture_model.pkl"
try:
    with open(model_filename, 'rb') as f:
        model = joblib.load(f)
except OSError as e:
    print("Error:", e)
    print("File:", model_filename)
    print("File exists:", os.path.exists(model_filename))
    exit()

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2)
imgSize = 300
offset = 20
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
fontColor = (255, 255, 255)
lineType = 2

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        if imgCrop.size != 0:  # Check if imgCrop is not empty
            try:
                # Resize the cropped image to match the input size expected by the model
                imgResize = cv2.resize(imgCrop, (150, 150))  # Assuming the model expects 150x150 images

                # Flatten the resized image to create a 1D array
                input_data = imgResize.flatten().reshape(1, -1)

                # Make prediction using the RandomForestClassifier model
                prediction = model.predict(input_data)[0]

                # Display the predicted action label near the hand region
                cv2.putText(img, prediction, (x, y - 10), font, fontScale, fontColor, lineType)
            except cv2.error as e:
                print(f"OpenCV Error: {e}")
            except Exception as e:
                print(f"Error: {e}")

    # Display the output image
    cv2.imshow("Image", img)
    
    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 